In [2]:
#  Install dependencies
!pip install gradio scikit-learn pandas numpy plotly streamlit-lottie

import gradio as gr
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import datetime
import plotly.graph_objects as go
import json
import requests


def load_lottieurl(url):
    r = requests.get(url)
    if r.status_code != 200:
        return None
    return r.json()


healthy_gif_url = "https://lottie.host/9b5cf18d-6eb9-4e5f-b90b-0aa3f1932a8f/H9FoPvbYFV.json"
detect_gif_url = "https://lottie.host/f9f34f1b-7b20-4061-8a83-ec8e9b39ce9e/UvW7FAeqDL.json"


np.random.seed(42)
remedies = [
    "Exercise regularly, manage weight, reduce sugar",
    "Follow a balanced diet, practice yoga",
    "Take prescribed medication, improve sleep cycle",
    "Maintain healthy weight, reduce stress"
]

diet_tips = [
    "Try flaxseeds in your smoothies",
    "Cut down on processed sugar",
    "Add leafy greens to every meal",
    "Include Omega-3s from walnuts and fish"
]

yoga_poses = [
    "Bhujangasana (Cobra Pose)",
    "Setu Bandhasana (Bridge Pose)",
    "Baddha Konasana (Butterfly Pose)",
    "Malasana (Garland Pose)"
]

life_tips = [
    "Stay positive & active",
    "Daily walk + hydration",
    "Track your periods",
    "Avoid processed food"
]

motivation_quotes = [
    "Your health is your wealth 💪",
    "Every step counts! Keep going!",
    "You are stronger than you think!",
    "Healing starts with small habits."
]

congrats_messages = [
    "🎉 Congratulations! You're healthy!",
    "✨ Keep shining and stay healthy!",
    "💖 Great job taking care of your health!",
    "🎊 You’re all good! Keep up the good lifestyle."
]

daily_tips = [
    "💧 Stay hydrated - aim for 8 glasses daily!",
    "🧘 Practice deep breathing for 5 mins today.",
    "🚪‍♀️ Take a 15-minute walk post meals.",
    "🥗 Include more fiber in your lunch."
]

df = pd.DataFrame({
    "Age": np.random.randint(18, 40, 500),
    "Weight": np.random.randint(45, 90, 500),
    "BMI": np.round(np.random.uniform(18, 35, 500), 1),
    "Pulse": np.random.randint(60, 110, 500),
    "LH/FSH Ratio": np.round(np.random.uniform(0.5, 2.5, 500), 2),
    "Follicle Count (Right)": np.random.randint(1, 25, 500),
    "PCOS": np.random.choice([0, 1], 500, p=[0.7, 0.3]),
    "Remedy": np.random.choice(remedies, 500),
    "Life Tip": np.random.choice(life_tips, 500),
    "Quote": np.random.choice(motivation_quotes, 500)
})

#  Model Training
X = df[["Age", "Weight", "BMI", "Pulse", "LH/FSH Ratio", "Follicle Count (Right)"]]
y = df["PCOS"]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
model = RandomForestClassifier()
model.fit(X_scaled, y)
y_pred = model.predict(X_scaled)
model_accuracy = round(accuracy_score(y, y_pred) * 100, 2)

summary_data = {}

def predict_pcos(name, age, weight, bmi, pulse, lh_fsh, follicles, acne, hairloss, irregular):
    input_data = pd.DataFrame([[age, weight, bmi, pulse, lh_fsh, follicles]],
                              columns=["Age", "Weight", "BMI", "Pulse", "LH/FSH Ratio", "Follicle Count (Right)"])
    scaled = scaler.transform(input_data)
    prob = model.predict_proba(scaled)[0][1] * 100
    pred = model.predict(scaled)[0]
    now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    symptoms = []
    if acne: symptoms.append("Acne")
    if hairloss: symptoms.append("Hair Loss")
    if irregular: symptoms.append("Irregular Periods")
    sym_str = ", ".join(symptoms) if symptoms else "None"

    yoga = np.random.choice(yoga_poses)
    diet = np.random.choice(diet_tips)

    fig = go.Figure()
    fig.add_trace(go.Scatterpolar(
        r=[age, weight, bmi, pulse, lh_fsh*10, follicles],
        theta=["Age", "Weight", "BMI", "Pulse", "LH/FSH x10", "Follicles"],
        fill='toself', name='Your Stats'))
    fig.update_layout(polar=dict(radialaxis=dict(visible=True)), showlegend=False)

    summary = f"{now}\nName: {name}\nResult: {'PCOS Detected' if pred==1 else 'No PCOS'}\nSymptoms: {sym_str}\nYoga: {yoga}\nDiet: {diet}"
    summary_data[name] = summary

    if pred == 1:
        remedy = np.random.choice(remedies)
        tip = np.random.choice(life_tips)
        quote = np.random.choice(motivation_quotes)
        return f"""
        <h3 style='color:red;'>⚠️ PCOS Detected!</h3>
        <p><b>Risk Probability:</b> {round(prob, 2)}%<br>
        <b>Remedy:</b> {remedy}<br><b>Symptom(s):</b> {sym_str}<br>
        <b>Yoga:</b> {yoga}<br><b>Diet:</b> {diet}<br>
        <b>Tip:</b> {tip}<br><i>{quote}</i><br><br>📊 <b>Model Accuracy:</b> {model_accuracy}%</p>
        """, fig
    else:
        congrats = np.random.choice(congrats_messages)
        return f"""
        <h3 style='color:green;'>✅ No PCOS Detected</h3>
        <p>{congrats}<br><b>Risk Probability:</b> {round(prob, 2)}%<br>
        <b>Tip:</b> {np.random.choice(daily_tips)}<br><br>📊 <b>Model Accuracy:</b> {model_accuracy}%</p>
        """, fig

def get_summary(name):
    return summary_data.get(name, "No summary yet. Please predict first.")

def next_period_date(last_date, cycle_length):
    try:
        last = pd.to_datetime(last_date)
        next_date = last + pd.Timedelta(days=int(cycle_length))
        return str(next_date.date())
    except:
        return "Invalid input."

def random_tip():
    return np.random.choice(daily_tips)

#  Gradio App
with gr.Blocks(theme=gr.themes.Soft()) as app:
    gr.Markdown("""
    <h1 style='text-align:center;color:#10b981;'>🌟 SheBalance: A PCOS Detection & Awareness Tool</h1>
    <p style='text-align:center;'>With risk prediction, visuals, tips, animations and health summary reports</p>
    """)

    gr.Markdown("""
    <center>
    <b> Real-time Health Scan Animation</b><br>
    <iframe src="https://lottie.host/embed/f9f34f1b-7b20-4061-8a83-ec8e9b39ce9e/UvW7FAeqDL.json" style="width:300px; height:300px; border:none;"></iframe>
    </center>
    """)

    with gr.Accordion("📈 Optional Cycle Tracker"):
        with gr.Row():
            last_period = gr.Textbox(label="Last Period Date (YYYY-MM-DD)")
            cycle = gr.Number(label="Cycle Length (days)", value=28)
            cycle_btn = gr.Button("🗖️ Predict Next Period")
            cycle_output = gr.Textbox(label="Next Expected Period")
            cycle_btn.click(fn=next_period_date, inputs=[last_period, cycle], outputs=cycle_output)

    with gr.Row():
        with gr.Column():
            name = gr.Textbox(label="Your Name")
            age = gr.Slider(18, 40, value=25, label="Age")
            weight = gr.Slider(45, 100, value=60, label="Weight")
            bmi = gr.Slider(18.0, 40.0, step=0.1, value=23.5, label="BMI")
        with gr.Column():
            pulse = gr.Slider(60, 120, value=75, label="Pulse")
            lh_fsh = gr.Slider(0.5, 2.5, step=0.1, value=1.2, label="LH/FSH Ratio")
            follicles = gr.Slider(1, 30, value=10, label="Follicles (Right Ovary)")

    with gr.Row():
        acne = gr.Checkbox(label="Acne")
        hairloss = gr.Checkbox(label="Hair Thinning")
        irregular = gr.Checkbox(label="Irregular Periods")

    predict_btn = gr.Button("🔍 Predict PCOS")
    result = gr.HTML()
    chart = gr.Plot()
    predict_btn.click(fn=predict_pcos, inputs=[name, age, weight, bmi, pulse, lh_fsh, follicles, acne, hairloss, irregular], outputs=[result, chart])

    with gr.Row():
        summary_btn = gr.Button("📄 Download Health Summary")
        summary_box = gr.Textbox(label="Summary", lines=6)
        summary_btn.click(fn=get_summary, inputs=name, outputs=summary_box)

    with gr.Accordion("📘 PCOS Info (Click to Expand)"):
        gr.Markdown("""
        - Polycystic Ovary Syndrome is a hormonal disorder.
        - Common signs include irregular periods, excess hair, and acne.
        - Early diagnosis and lifestyle changes can help manage it.
        """)

    tip_btn = gr.Button("💡 Daily Health Tip")
    tip_output = gr.Textbox(label="Tip")
    tip_btn.click(fn=random_tip, outputs=tip_output)

    gr.Markdown("<hr><center><i>Made with ❤️ for health awareness</i><br><b>Made by Purnika 💕</b></center>")


app.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://dfb1a6b52a0b125fc0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
